In [ ]:
import math
import numpy as np

#everything below is defining activation functions
#--------------------------------------------------------------------------------------------

#def relu(input):
  #/return max((0, max(input)))

def d_relu(input):
  if(input < 0 or input == 0):
    return 0
  else:
    return 1

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

def d_sigmoid(input):
  return sigmoid(input) * (1 - sigmoid(input))

def tanh(input):
  top = (math.exp(input) - math.exp(-input))
  bottom = (math.exp(input) + math.exp(-input))
  return (top/bottom)

#helper functions for tanh
def cosh(input):
  return ((math.exp(input) + math.exp(-input)) / 2)
def sinh(input):
  return ((math.exp(input) - math.exp(-input)) / 2) 

def d_tanh(input):
  top = (math.pow(cosh(input), 2) - math.pow(sinh(input), 2))
  bottom = math.pow(input, 2)
  return (top / bottom)

def softmax(z):
  # subracting the max adds numerical stability
  shiftx = z - np.max(z,axis=1)[:,np.newaxis]
  exps = np.exp(shiftx)
  return exps / np.sum(exps,axis=1)[:,np.newaxis]

def d_softmax(Y_hat, Y):
  return Y_hat - Y

def linear(input):
  return input
def d_linear(input):
  return 1



def d_activate(input_X, activation_function):
  if(activation_function.lower() == 'relu'):
    return d_relu(input_X)
  elif(activation_function.lower() == 'sigmoid'):
    return d_sigmoid(input_X)
  elif(activation_function.lower() == 'tanh'):
    return d_tanh(input_X)
  elif(activation_function.lower() == 'softmax'):
    return d_softmax(input_X)
  elif(activation_function.lower() == 'linear'):
    return d_linear(input_X)
  else:
    print("That activation function is not defined!")
    return None

def activate(input_X, activation_function):
  if(activation_function.lower() == 'relu'):
    return relu(input_X)
  elif(activation_function.lower() == 'sigmoid'):
    return sigmoid(input_X)
  elif(activation_function.lower() == 'tanh'):
    return tanh(input_X)
  elif(activation_function.lower() == 'softmax'):
    return softmax(input_X)
  elif(activation_function.lower() == 'linear'):
    return linear(input_X)
  else:
    print("That activation function is not defined!")
    return None

In [ ]:
class Model:
  def __init__(self, batch, lr):
    #initializing self lists to keep track of stuff for bacthes, forward prop & backporp
    self.batch = batch
    self.lr = lr
    self.W = []
    self.B = []
    self.A = []
    self.Z = []
    self.X = []
    self.layers = 0
    self.tempW = []
    self.tempB = []
    self.initialized = False

    #store error for backprop
    self.output_error = []
  
  #initialize the weights during 'model.add' so we can test our network shapes dynamically w/out model.compile
  #added an output bool here so we can make sure the shape of the output network is (1,n)
  def initial_weights(self, input_data, output_shape):
    B = np.zeros((1, output_shape))
    #assigning the shape 
    W = np.random.uniform(-1e-3, 1e-3, size = (input_data.shape[len(input_data.shape) - 1], output_shape))
    self.B.append(B)
    self.W.append(W)

  def add(self, input_data, output_shape, activation, layer_number, final_layer):
    #checking to see if we already initialized our model
    
    #append to layers so we have a correct index value
    index = layer_number

    #making sure our data in a numpy array
    if (type(input_data) == np.ndarray):
      X = input_data
    else:
      X = np.asarray(input_data)

    if (self.initialized == False):
      #adding data and activations to self lists
      self.X.append(X)
      self.A.append(activation)

      #keep track of our index & initializing random weights for dynamic comatibility testing
      self.initial_weights(input_data, output_shape)

      X2 = self.forward(input_data, index)
      #printing layer info 
      print("Layer:", index)
      print("Input Shape: ", X.shape)
      print("Weight Shape: ", self.W[index].shape)
      print("Output Shape: ", X2.shape)
      print(" ")
      self.layers = self.layers + 1
      if (final_layer == True):
        self.initialized = True
      return(X2)
    else:
      X2 = self.forward(input_data, index)
      return(X2)
    
    
  
  def forward(self, input_data, index):
    #pulling weights and biases from  main lists for operations
    B = self.B[index]
    W = self.W[index]

    #matmul of data # weights + bias
    Z = np.matmul(input_data, W) + B

    #pulling activation from index 
    act = str(self.A[index])
    #activating 
    Z = activate(Z, act)
    #keeping track of Z i guess
    self.Z.append(Z)
    return(Z)

  
  def backprop(self, model_output):
    model_output = model_output
    for i in range(len(self.layers)):
      i = self.layers - i
      act = str(self.A[i])
      Z = self.Z[i]
      model_output = np.transpose(d_activate(Z, act)) * model_output
      self.W[i] = self.W[i] - self.lr*model_output
      self.B[i] = self.B[i] - self.lr*model_output
      



In [ ]:
model = Model(128, 0.01)

X = np.array([[1,3], [3,2], [3,3]])
print(X.shape)
Y = np.array([4, 5, 6])
print(Y.shape)

Z = model.add(X, 2, "linear", 0, False)
Z = model.add(Z, 4, "linear", 1, False)
Z = model.add(Z, 5, "softmax", 2, True)
Z.shape


(3, 2)
(3,)
Layer: 0
Input Shape:  (3, 2)
Weight Shape:  (2, 2)
Output Shape:  (3, 2)
 
Layer: 1
Input Shape:  (3, 2)
Weight Shape:  (2, 4)
Output Shape:  (3, 4)
 
Layer: 2
Input Shape:  (3, 4)
Weight Shape:  (4, 5)
Output Shape:  (3, 5)
 


(3, 5)